In [1]:
from __future__ import absolute_import
from __future__ import division
from __future__ import print_function

import time

from six.moves import xrange  # pylint: disable=redefined-builtin
import tensorflow as tf

import TataSteelChemistryModel as model
# import IndianPines_data_set as input_data
import TataSteel as input_data

In [2]:

learning_rate = 0.001
max_steps = 10000
hidden1 = 500
hidden2 = 300
batch_size = 1000
train_dir = '/users/TeamSteel/TATA_Steel_Project/TATA_Steel_Project/Data'

In [3]:
def placeholder_inputs(batch_size):
    """Generate placeholder variables to represent the input tensors.
    These placeholders are used as inputs by the rest of the model building
    code and will be fed from data in the .run() loop, below.
    Args:
    batch_size: The batch size will be baked into both placeholders.
    Returns:
    images_placeholder: Images placeholder.
    targets_placeholder: Targets placeholder.
    """
    # Note that the shapes of the placeholders match the shapes of the full
    # image and target tensors, except the first dimension is now batch_size
    # rather than the full size of the train or test data sets.
    inputs_placeholder = tf.placeholder(tf.float32, shape=(batch_size, model.NUM_INPUT_FEATURES))
    targets_placeholder = tf.placeholder(tf.float32, shape=(batch_size, model.NUM_TARGET_ATTRIBUTES))
    return inputs_placeholder, targets_placeholder


In [4]:
def fill_feed_dict(data_set, inputs_pl, targets_pl):
    """Fills the feed_dict for training the given step.
    A feed_dict takes the form of:
    feed_dict = {
      <placeholder>: <tensor of values to be passed for placeholder>,
      ....
    }
    Args:
    data_set: The set of images and labels, from input_data.read_data_sets()
    inputs_pl: The images placeholder, from placeholder_inputs().
    targets_pl: The labels placeholder, from placeholder_inputs().
    Returns:
    feed_dict: The feed dictionary mapping from placeholders to values.
    """
    # Create the feed_dict for the placeholders filled with the next
    # `batch size ` examples.
    inputs_feed, targets_feed = data_set.next_batch(batch_size)
    feed_dict = {
      inputs_pl: inputs_feed,
      targets_pl: targets_feed,
    }
    return feed_dict

In [5]:
def do_eval(sess,
            eval_MSE,
            eval_RMSE_ratio_vec,
            inputs_placeholder,
            targets_placeholder,
            data_set):
    """Runs one evaluation against the full epoch of data.
    Args:
    sess: The session in which the model has been trained.
    eval_MSE: The Tensor that returns the MSE.
    inputs_placeholder: The images placeholder.
    targets_placeholder: The targets placeholder.
    data_set: The set of inputs and targets to evaluate, from
      input_data.read_data_sets().
    """
    # And run one epoch of eval.
    squared_err = 0 
    root_batchwise_mean_squared_err_ratio = 0
    steps_per_epoch = data_set.num_examples // batch_size
    num_examples = steps_per_epoch * batch_size
    for step in xrange(steps_per_epoch):
        feed_dict = fill_feed_dict(data_set,
                                   inputs_placeholder,
                                   targets_placeholder)
        squared_err += sess.run(eval_MSE, feed_dict=feed_dict)
        root_batchwise_mean_squared_err_ratio += sess.run(eval_RMSE_ratio_vec, feed_dict=feed_dict)
    
    mean_squared_err = squared_err / steps_per_epoch
    root_mean_squared_err_ratio = root_batchwise_mean_squared_err_ratio  / steps_per_epoch
    print('  Num examples: %d  MSE: %0.04f' %
        (num_examples, mean_squared_err))
    print('  Target-wise RMSE-ratio: ', root_mean_squared_err_ratio)


In [6]:
def run_training():
    """Train TataSteelModel for a number of steps."""
    # Get the sets of inputs and targets for training, validation, and
    # test on TataSteel.
    
    data_sets = input_data.read_data_sets(train_dir,'chemistry_input_data.pkl')
    Training_data = data_sets.train
#     print(Training_data._inputs.shape, Training_data._targets.shape)
    Validation_data = data_sets.validation
    Test_data = data_sets.test
        
    # Tell TensorFlow that the model will be built into the default Graph.
    with tf.Graph().as_default():
    # Generate placeholders for the images and labels.
        inputs_placeholder, targets_placeholder = placeholder_inputs(batch_size)

        # Build a Graph that computes predictions from the inference model.
        logits = model.inference(inputs_placeholder, hidden1, hidden2)

        # Add to the Graph the Ops for loss calculation.
        loss = model.loss(logits, targets_placeholder)

        # Add to the Graph the Ops that calculate and apply gradients.
        train_op = model.training(loss, learning_rate)

        # Add the Op to compare the logits to the labels during evaluation.
        eval_RMSE_ratio_vec, eval_MSE = model.evaluation(logits, targets_placeholder)

        # Build the summary operation based on the TF collection of Summaries.
    #    summary_op = tf.merge_all_summaries()

        # Add the variable initializer Op.
        init = tf.initialize_all_variables()

        # Create a saver for writing training checkpoints.
        saver = tf.train.Saver()

        # Create a session for running Ops on the Graph.
        sess = tf.Session()

        # Instantiate a SummaryWriter to output summaries and the Graph.
    #    summary_writer = tf.train.SummaryWriter(FLAGS.train_dir, sess.graph)

        # And then after everything is built:

        # Run the Op to initialize the variables.
        sess.run(init)

        # Start the training loop.
        for step in xrange(max_steps):
            start_time = time.time()

            # Fill a feed dictionary with the actual set of images and labels
            # for this particular training step.
            feed_dict = fill_feed_dict(Training_data,
                                     inputs_placeholder,
                                     targets_placeholder)

            # Run one step of the model.  The return values are the activations
            # from the `train_op` (which is discarded) and the `loss` Op.  To
            # inspect the values of your Ops or variables, you may include them
            # in the list passed to sess.run() and the value tensors will be
            # returned in the tuple from the call.
            _, loss_value, logit, mse =sess.run([train_op, loss, logits, eval_MSE],
                                   feed_dict=feed_dict) 

            duration = time.time() - start_time

            # Write the summaries and print an overview fairly often.
            if step % 100 == 0:
            # Print status to stdout.
                print('Step %d: loss = %.2f (%.3f sec)' % (step, loss_value, duration))
                #print (logit)
            # Update the events file.
    #             summary_str = sess.run(summary_op, feed_dict=feed_dict)
    #             summary_writer.add_summary(summary_str, step)
    #             summary_writer.flush()

            # Save a checkpoint and evaluate the model periodically.
            if (step + 1) % 1000 == 0 or (step + 1) == max_steps:
                saver.save(sess, 'chemistry-model-new.ckpt', global_step=step)

            # Evaluate against the training set.
                print('Training Data Eval:')
                do_eval(sess,
                        eval_MSE,
                        eval_RMSE_ratio_vec,
                        inputs_placeholder,
                        targets_placeholder,
                        Training_data)
                print('Validation Data Eval:')
                do_eval(sess,
                        eval_MSE,
                        eval_RMSE_ratio_vec,
                        inputs_placeholder,
                        targets_placeholder,
                        Validation_data)
                print('Test Data Eval:')
                do_eval(sess,
                        eval_MSE,
                        eval_RMSE_ratio_vec,
                        inputs_placeholder,
                        targets_placeholder,
                        Test_data)


In [7]:
run_training()

ResourceExhaustedError: OOM when allocating tensor with shape[1000,500]
	 [[Node: h_FC1/MatMul = MatMul[T=DT_FLOAT, transpose_a=false, transpose_b=false, _device="/job:localhost/replica:0/task:0/gpu:0"](_recv_Placeholder_0/_11, h_FC1/weights/read)]]
	 [[Node: gradients/l2_loss_mean_grad/Shape_2/_13 = _Recv[client_terminated=false, recv_device="/job:localhost/replica:0/task:0/cpu:0", send_device="/job:localhost/replica:0/task:0/gpu:0", send_device_incarnation=1, tensor_name="edge_184_gradients/l2_loss_mean_grad/Shape_2", tensor_type=DT_INT32, _device="/job:localhost/replica:0/task:0/cpu:0"]()]]
Caused by op u'h_FC1/MatMul', defined at:
  File "/usr/lib/python2.7/runpy.py", line 162, in _run_module_as_main
    "__main__", fname, loader, pkg_name)
  File "/usr/lib/python2.7/runpy.py", line 72, in _run_code
    exec code in run_globals
  File "/usr/local/lib/python2.7/dist-packages/ipykernel/__main__.py", line 3, in <module>
    app.launch_new_instance()
  File "/usr/local/lib/python2.7/di